In [1]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [2]:
train_csv = pd.read_csv('train.csv', index_col=0)

In [3]:
#stopword removal and lemmatization
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [4]:
train_csv.head()
train_csv.count()

labels    37554
text      37554
dtype: int64

In [5]:
Train = train_csv.iloc[:30000]
Test = train_csv.iloc[30001:]

In [7]:
Train.count()

labels    30000
text      30000
dtype: int64

In [8]:
# Train test split - Splitting train and test dataset into 80:20
# X_train_non, X_test_non, y_train, y_test = train_test_split(train_csv['text'], train_csv['labels'], test_size = 0.20)
X_train_non = Train['text']
X_test_non = Test['text']
y_train = Train['labels']
y_test = Test['labels']

X_train = []
X_test = []

In [13]:
X_train_non[5]

'The White House correspondent for Playboy Magazine—an adult entertainment publication—sprung up during Tuesday ’ s White House briefing to airbrush the embattled CNN ’ s reputation as the very fake news scandal embroiling the network received harsh treatment from Deputy White House Press Secretary Sarah Huckabee Sanders .\nAfter ███ ’ s senior White House correspondent Charlie Spiering asked the White House for the president ’ s thoughts on the very fake news scandal ripping CNN apart from the inside out . Playboy Magazine sprung to CNN ’ s defense .\nAs Sanders hammered the media for its failures in the wake of the very fake news scandal ’ s latest developments , Playboy ’ s Brian Karem jumped in from the side of the briefing room to back up CNN . Karem said :\nYou ’ re inflaming everybody right here right now with those words . This administration has done that as well . Why in the name of heaven ? Any one of us , right , are replaceable . And any one of us , if we don ’ t get it ri

In [14]:
#text pre processing
for i in range(0, len(X_train_non)):
    news_text = re.sub("[^a-zA-Z]", " ", str(X_train_non[i]))
    news_text = news_text.lower()
    news_text = news_text.split()
    news_text = [lemmatizer.lemmatize(word) for word in news_text if not word in set(stopwords)]
    news_text = ' '.join(news_text)
    X_train.append(news_text)

In [27]:
#text pre processing
for i in range(30001, 30000+len(X_test_non)):
    news_text = re.sub('[^a-zA-Z]', ' ', X_test_non[i])
    news_text = news_text.lower()
    news_text = news_text.split()
    news_text = [lemmatizer.lemmatize(word) for word in news_text if not word in set(stopwords)]
    news_text = ' '.join(news_text)
    X_test.append(news_text)

In [16]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(X_train)
#applying tf idf to training data
X_train_tf = tf_idf.transform(X_train)

In [17]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 30000, n_features: 104037


In [18]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(X_test)
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 7552, n_features: 104037


In [19]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)

MultinomialNB()

In [20]:
#predicted y
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [25]:
print(metrics.classification_report(y_test[:7552], y_pred))

              precision    recall  f1-score   support

           0       0.51      0.66      0.57      2595
           1       0.71      0.16      0.26      2172
           2       0.53      0.71      0.61      2785

    accuracy                           0.53      7552
   macro avg       0.58      0.51      0.48      7552
weighted avg       0.58      0.53      0.49      7552

